In [ ]:
! pip install langchain-anthropic langgraph

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("ANTHROPIC_API_KEY")

## Memory Profile

You can think of this as a fixed profile of facts ("semantic" memory) about the user.

These are typically user attributes that fall within a pre-defined schema. 

You'll see that this gives us a useful, general context to have in all of our interactions with the user.

Let's build on what we did in the previous lessons.

First, we define a profile schema for the user.

![](./img/memory_course_semantic_profile.png)


In [1]:
from pydantic import BaseModel, Field
from typing import List
from langchain_anthropic import ChatAnthropic

# Schema for user profile
class UserProfile(BaseModel):
    name: str = Field(None, description="User's full name")
    location: str = Field(None, description="User's primary location/city")
    workplace: str = Field(None, description="User's place of work")
    relationships: list[dict] = Field(None, description="User's relationships")

# Define and augment LLM with structured output
llm = ChatAnthropic(model="claude-3-5-sonnet-latest")
structured_llm = llm.with_structured_output(UserProfile)

In [2]:
# We can store the instructions for easy access and updating
from langgraph.store.memory import InMemoryStore
in_memory_store = InMemoryStore()

In [14]:
import sys
sys.path.append('..')
import src.memory_course.prompts
import src.memory_course.examples
import src.memory_course.utils

# Reload the module in case any changes were made
import importlib
importlib.reload(src.memory_course.prompts)
importlib.reload(src.memory_course.examples)
importlib.reload(src.memory_course.utils)

# Then import the instructions and examples
from src.memory_course.utils import format_few_shot_examples
from src.memory_course.examples import example_input, example_output
from src.memory_course.prompts import memory_collection_extraction_instructions, memory_search_instructions, memory_profile_extraction_instructions, profile_extraction_input

In [5]:
# Profile extraction instructions
namespace = ("journal","instructions")
key = "instructions_profile_extraction"
in_memory_store.put(namespace, key, {"instructions": memory_profile_extraction_instructions})

In [10]:
def extract_profile(journal_entry):
    """Extract a profile from a journal entry.
    
    Args:
        journal_entry (str): The text of the journal entry to analyze
        
    Returns:
        dict: A dictionary containing extracted profile
    """
    # Get instructions
    memory_profile_extraction_instructions = in_memory_store.get(("journal","instructions"), "instructions_profile_extraction")

    # Get existing profile
    namespace = ("journal", "memory", "profile")
    key = "profile"
    profile = in_memory_store.get(namespace, key)

    # Format input
    input = profile_extraction_input.format(
        memory_profile_extraction_instructions=memory_profile_extraction_instructions,
        profile=profile,
        journal_entry=journal_entry
    )

    # Run extraction
    profile = structured_llm.invoke(input)

    return profile

def store_user_profile(profile):
    """Store the user's profile in the memory store.
        
    Args:
        profile (dict): The profile information to store, containing extracted
            personality traits, preferences, and characteristics
            
    Returns:
        None
    """
    # Iterate through each memory and store in appropriate collection
    namespace = ("journal", "memory", "profile")
    key = "profile"
    value = profile
    in_memory_store.put(namespace, key, value)

In [11]:
# Test extraction of memories
journal_entry = """
I'll first introduce myself before I start the journal.

I'm Lance, I work at LangChain, I live in San Francisco, and I'm married with a 16 month old daughter.
"""

profile=extract_profile(journal_entry)
profile

UserProfile(name='Lance', location='San Francisco', workplace='LangChain', relationships=[{'type': 'spouse', 'description': 'married'}, {'type': 'daughter', 'description': '16 months old'}])

In [12]:
store_user_profile(profile)

In [13]:
# Test updating profile
journal_entry = """
I need to ping Will (co-worker) about the new LangChain memory docs.
"""

profile=extract_profile(journal_entry)
profile

UserProfile(name='Lance', location='San Francisco', workplace='LangChain', relationships=[{'type': 'spouse', 'description': 'married'}, {'type': 'daughter', 'description': '16 months old'}, {'type': 'co-worker', 'description': 'Will'}])

So now we can:

1. Extract memories (collections) and user profile from a journal entry given a set of instructions (system message)
2. Update those instructions based upon user feedback directly 
3. Add few shot examples to improve the quality of the extraction
4. Update those few shot examples based upon user feedback directly 
5. Save a collection of extracted memories to the store
6. Search for memories in a given collection in natural language

This is a simple example of using and editing both "procedural memory" (system prompt/instructions) and "episodic memory" (few shot examples). 

In addition, we save "semantic memories" (fact collections as well as a single profile) about the user.

And we can search for memories in a given collection in natural language. 
